In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# Configurações iniciais
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
num_classes = 10  # CIFAR-10 tem 10 classes

# Transformações para o conjunto de dados ImageNet e CIFAR-10
transform_imagenet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_cifar10 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# Carregar o conjunto de dados ImageNet (usando mini-batch amostral como exemplo)
# Normalmente o ImageNet completo não é publicamente disponível de forma gratuita,
# mas aqui mostramos como seria feito para um subconjunto
imagenet_data = datasets.FakeData(transform=transform_imagenet)  # Exemplo com dados simulados
imagenet_loader = DataLoader(imagenet_data, batch_size=batch_size, shuffle=True)

# Carregar o modelo pré-treinado ResNet18 para ImageNet
model = models.resnet18(pretrained=True)
model = model.to(device)
model.eval()

# Inferência no ImageNet (ou subconjunto simulado)
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in imagenet_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Acurácia no conjunto de dados ImageNet: {100 * correct / total:.2f}%")

# Parte 2: Transfer Learning para CIFAR-10
# Carregar conjunto de dados CIFAR-10
cifar10_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar10)
cifar10_loader = DataLoader(cifar10_data, batch_size=batch_size, shuffle=True)

# Ajustar a camada final do modelo para CIFAR-10
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Definir o otimizador e a função de perda
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinar o modelo com CIFAR-10
num_epochs = 5  # Ajuste para o número de épocas desejado
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in cifar10_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Época [{epoch+1}/{num_epochs}], Loss: {running_loss / len(cifar10_loader):.4f}")

# Avaliação no conjunto de teste CIFAR-10
cifar10_test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar10)
cifar10_test_loader = DataLoader(cifar10_test_data, batch_size=batch_size, shuffle=False)

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in cifar10_test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Acurácia no conjunto de teste CIFAR-10: {100 * correct / total:.2f}%")


/home/lorran/github/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lorran/github/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Acurácia no conjunto de dados ImageNet: 0.00%


100%|██████████| 170M/170M [00:42<00:00, 3.99MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data


KeyboardInterrupt: 